# ChatFlow Example

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os, json, copy

from colink import CoLink

from aiflows.utils import serve_utils
from aiflows.utils.general_helpers import read_yaml_file
from aiflows.messages import FlowMessage
from aiflows.utils import coflows_utils, colink_utils
from aiflows.workers import run_dispatch_worker_threads, run_dispatch_worker_thread
from aiflows import flow_verse

dependencies = [
    {"url": "aiflows/ChatFlowModule", "revision": "coflows"},
]
flow_verse.sync_dependencies(dependencies)

[2024-03-17 03:49:00,946][aiflows.flow_verse.loading:775][INFO] - [<interactive>] started to sync flow module dependencies to /home/staverm/workspace/coflows-dev/aiflows/examples/mauro_examples/ChatFlow-languages/flow_modules...
[2024-03-17 03:49:01,095][aiflows.flow_verse.loading:608][INFO] - aiflows/ChatFlowModule:coflows already synced, skip
[2024-03-17 03:49:01,097][aiflows.flow_verse.loading:825][INFO] - [<interactive>] finished syncing




['/home/staverm/workspace/coflows-dev/aiflows/examples/mauro_examples/ChatFlow-languages/flow_modules/aiflows/ChatFlowModule']

# Connect to CoLink Server

In [3]:
cl = colink_utils.start_colink_server()

# Start a few default workers

In [4]:
run_dispatch_worker_thread(cl)

Dispatch worker started in attached thread.
dispatch_point: coflows_dispatch

~~~ Dispatch task ~~~
flow_endpoint: Chatbot
flow_id: 469b3425-87d6-4a58-9ae6-96ee6f7b299a
message_paths: ['push_tasks:1cb5ce64-354a-4bfb-a3fc-0bbee5c09349:msg']
parallel_dispatch: False


~~~ Dispatch task ~~~
flow_endpoint: Chatbot
flow_id: 469b3425-87d6-4a58-9ae6-96ee6f7b299a
message_paths: ['push_tasks:78046133-4082-4f06-bf4e-2c0a74bc69bd:msg']
parallel_dispatch: False

[2024-03-17 03:49:09,380][aiflows.utils.general_helpers:387][INFO] -  

For feedback or to get help:  https://github.com/epfl-dlab/aiflows/issues 


[2024-03-17 03:49:09,382][aiflows.utils.general_helpers:397][ERROR] - 'query'
Traceback (most recent call last):
  File "/home/staverm/workspace/coflows-dev/aiflows/aiflows/utils/general_helpers.py", line 406, in wrapper
    return f(*args, **kw)
  File "/home/staverm/workspace/coflows-dev/aiflows/aiflows/base_flows/abstract.py", line 535, in __call__
    self._run_method(input_message)
  File

Exception in thread Thread-5 (run):
Traceback (most recent call last):
  File "/home/staverm/.pyenv/versions/3.10.6/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/home/staverm/.pyenv/versions/3.10.6/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 761, in run_closure
    _threading_Thread_run(self)
  File "/home/staverm/.pyenv/versions/3.10.6/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/home/staverm/workspace/coflows-dev/colink-sdk-python-dev/colink/protocol.py", line 128, in run
    raise err
  File "/home/staverm/workspace/coflows-dev/colink-sdk-python-dev/colink/protocol.py", line 20, in thread_func
    cl_app.start()
  File "/home/staverm/workspace/coflows-dev/colink-sdk-python-dev/colink/protocol.py", line 220, in start
    raise e
  File "/home/staverm/workspace/coflows-dev/colink-sdk-python-dev/colink/protocol.py", line 213, in start
    self.user_func(cl, task.protocol_param,

In [5]:
run_dispatch_worker_thread(cl)

Dispatch worker started in attached thread.
dispatch_point: coflows_dispatch

~~~ Dispatch task ~~~
flow_endpoint: Chatbot
flow_id: 469b3425-87d6-4a58-9ae6-96ee6f7b299a
message_paths: ['push_tasks:8c80b814-7368-4f07-8a2b-d5b4a897f92b:msg']
parallel_dispatch: False



# Serve ChatFlow

In [6]:
serve_utils.serve_flow(
    cl=cl,
    flow_class_name="flow_modules.aiflows.ChatFlowModule.ChatAtomicFlow",
    flow_endpoint="Chatbot",
)

Started serving flow_modules.aiflows.ChatFlowModule.ChatAtomicFlow at flows:Chatbot.
dispatch_point: coflows_dispatch
parallel_dispatch: False
singleton: True


True

## Get two instances with different configurations

In [7]:
root_dir = "."
cfg_path = os.path.join(root_dir, "flow_modules/aiflows/ChatFlowModule/demo.yaml")
cfg = read_yaml_file(cfg_path)

In [8]:
args_A = copy.deepcopy(cfg)
args_A["system_message_prompt_template"]["template"] = \
"You are a helpful chatbot that truthfully answers questions. Answer in the language English."

chatbot_A = serve_utils.get_flow_instance(
    cl=cl,
    flow_endpoint="Chatbot",
    config_overrides=args_A,
)

Mounted 469b3425-87d6-4a58-9ae6-96ee6f7b299a at flows:Chatbot:mounts:local:469b3425-87d6-4a58-9ae6-96ee6f7b299a


In [9]:
args_B = copy.deepcopy(cfg)
args_B["system_message_prompt_template"]["template"] = \
"You are a helpful chatbot that truthfully answers questions. Answer in the language Croatian."


chatbot_B = serve_utils.get_flow_instance(
    cl=cl,
    flow_endpoint="Chatbot",
    config_overrides=args_B,
)

Fetched singleton 469b3425-87d6-4a58-9ae6-96ee6f7b299a


## Conversation with chatbot A

In [10]:
input_message = FlowMessage(
    data={"id": 0, "question": "What is the capital of Switzerland?"},
)

future = chatbot_A.get_reply_future(input_message)
print(future.get_data())

input_message = FlowMessage(
    data={"id": 0, "query": "Where is it located?"},
)

future = chatbot_A.get_reply_future(input_message)
print(future.get_data())

{'api_output': 'The capital of Switzerland is Bern.'}
{'api_output': 'Bern, the capital of Switzerland, is located in the central part of the country, on the Swiss Plateau.'}


## Conversation with chatbot B

In [ ]:
input_message = FlowMessage(
    data={"id": 0, "question": "What is the capital of Switzerland?"},
)

future = chatbot_B.get_reply_future(input_message)
print(future.get_data())

input_message = FlowMessage(
    data={"id": 0, "query": "Where is it located?"},
)

future = chatbot_B.get_reply_future(input_message)
print(future.get_data())